# Carefree Learn
https://medium.com/pytorch/carefree-learn-tabular-datasets-%EF%B8%8F-pytorch-e329b2f008f2

Note Carefree pulls in OpenCV and also has dependencies on Optuna and Pytorch that we might need to be careful with

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# Load Data
train = pd.read_feather("/home/jovyan/work/ray_experiments/data/train_feat_1_baseline.feather")
target = train['target'] 
subtrain = train.drop(['ID_code', 'target'], axis = 1)

In [3]:
# need to convert everything to numpy array first
X_train, X_test, y_train, y_test = train_test_split(subtrain, target, test_size=0.3, random_state=42)
data = X_train.to_numpy(), y_train.to_numpy(), X_test.to_numpy(), y_test.to_numpy()

# Fit cflearn basic

In [4]:
import cflearn
from cfdata.tabular import *

auto = cflearn.Auto("clf", models="fcnn", batch_size=32)
auto.fit(*data, num_jobs=0)

[I 2020-12-21 10:39:34,126] A new study created in memory with name: fcnn_optuna


~~~ [warning] '__tmp__/__optuna__/0/fcnn/1' already exists, it will be cleared up to store our logging


[W 2020-12-21 11:22:44,664] Trial 0 failed because of the following error: RuntimeError('CUDA out of memory. Tried to allocate 790.00 MiB (GPU 0; 10.76 GiB total capacity; 9.14 GiB already allocated; 74.81 MiB free; 9.18 GiB reserved in total by PyTorch)')
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/optuna/_optimize.py", line 189, in _run_trial
    value = func(trial)
  File "/opt/conda/lib/python3.8/site-packages/cflearn/api/hpo.py", line 964, in objective
    result = tuner.train(
  File "/opt/conda/lib/python3.8/site-packages/cflearn/api/hpo.py", line 186, in train
    repeat_result = repeat_with(
  File "/opt/conda/lib/python3.8/site-packages/cflearn/api/basic.py", line 407, in repeat_with
    local_pipelines.append(m.fit(x, y, x_cv, y_cv))
  File "/opt/conda/lib/python3.8/site-packages/cflearn/pipeline.py", line 526, in fit
    self._loop()
  File "/opt/conda/lib/python3.8/site-packages/cflearn/pipeline.py", line 288, in _loop
    self.trainer

RuntimeError: CUDA out of memory. Tried to allocate 790.00 MiB (GPU 0; 10.76 GiB total capacity; 9.14 GiB already allocated; 74.81 MiB free; 9.18 GiB reserved in total by PyTorch)

In [ ]:
predictions = auto.predict(X_test)
print("accuracy:", (y_test == predictions).mean())

In [ ]:
cflearn.evaluate(
        X_test,
        y_test,
        pipelines=fcnn,
        other_patterns={"auto": auto.pattern},
    )